In [1]:
pip install pulp

In [ ]:
import pulp

In [ ]:
departments = [
    "Ophthalmology",
    "Gynecology",
    "Oral Surgery",
    "Otolaryngology",
    "General Surgery"
]

days = ["Mon", "Tue", "Wed", "Thu", "Fri"]

In [ ]:
ORs_per_day = 10
hours_per_OR = 8

In [ ]:
# Total surgery teams available per day (n_k)
nk = {
    "Ophthalmology": {"Mon":2, "Tue":2, "Wed":2, "Thu":2, "Fri":2},
    "Gynecology": {"Mon":3, "Tue":3, "Wed":3, "Thu":3, "Fri":3},
    "Oral Surgery": {"Mon":0, "Tue":1, "Wed":0, "Thu":1, "Fri":0},
    "Otolaryngology": {"Mon":1, "Tue":1, "Wed":1, "Thu":1, "Fri":1},
    "General Surgery": {"Mon":6, "Tue":6, "Wed":6, "Thu":6, "Fri":6}
}

In [ ]:
# Weekly target hours
target_hours = {
    "Ophthalmology": 39.4,
    "Gynecology": 117.4,
    "Oral Surgery": 19.9,
    "Otolaryngology": 26.3,
    "General Surgery": 189
}

# Max ORs per department per day
max_daily = {
    "Ophthalmology": {"Mon":2, "Tue":2, "Wed":2, "Thu":2, "Fri":2},
    "Gynecology": {"Mon":3, "Tue":3, "Wed":3, "Thu":3, "Fri":3},
    "Oral Surgery": {"Mon":1, "Tue":1, "Wed":1, "Thu":1, "Fri":1},
    "Otolaryngology": {"Mon":1, "Tue":1, "Wed":1, "Thu":1, "Fri":1},
    "General Surgery": {"Mon":6, "Tue":6, "Wed":6, "Thu":6, "Fri":6}
}

# Weekly min / max
min_weekly = {
    "Ophthalmology": 3,
    "Gynecology": 12,
    "Oral Surgery": 2,
    "Otolaryngology": 2,
    "General Surgery": 18
}

max_weekly = {
    "Ophthalmology": 6,
    "Gynecology": 18,
    "Oral Surgery": 3,
    "Otolaryngology": 4,
    "General Surgery": 25
}

In [ ]:
model = pulp.LpProblem("OR_Scheduling", pulp.LpMaximize)

In [ ]:
# Decision variables
x = pulp.LpVariable.dicts(
    "ORs",
    [(j, k) for j in departments for k in days],
    lowBound=0,
    cat="Integer"
)


In [ ]:
#Objective function
model += pulp.lpSum(
    hours_per_OR * x[(j, k)] / target_hours[j]
    for j in departments for k in days
)

In [ ]:
#constraints
# (1) Physical OR capacity: At most 10 ORs are assigned every day
for k in days:
    model += pulp.lpSum(x[(j, k)] for j in departments) <= ORs_per_day

# (2) Surgery team capacity: The number of ORs allocated to a department on a
#given day cannot exceed the number of surgery
#teams that department has available that day
for j in departments:
    for k in days:
        model += x[(j, k)] <= nk[j][k]

# (3) Daily min / max per department: Meet department daily minimums and maximums
for j in departments:
    for k in days:
        model += x[(j, k)] <= max_daily[j][k]

# (4) Weekly min / max per department: Meet department weekly minimums and maximums
for j in departments:
    model += pulp.lpSum(x[(j, k)] for k in days) >= min_weekly[j]
    model += pulp.lpSum(x[(j, k)] for k in days) <= max_weekly[j]


In [ ]:
model.solve()

1

In [ ]:
import pandas as pd

data = {
    j: {k: int(x[(j, k)].value() or 0) for k in days}
    for j in departments
}

df = pd.DataFrame.from_dict(data, orient="index")
df.index.name = "Reparto"

print(df)


                 Mon  Tue  Wed  Thu  Fri
Reparto                                 
Ophthalmology      0    2    2    2    0
Gynecology         3    3    3    3    3
Oral Surgery       0    1    0    1    0
Otolaryngology     1    1    0    1    1
General Surgery    6    3    5    3    6


In [ ]:
print("\nObjective value:", pulp.value(model.objective))


Objective value: 5.234715731411526


In [ ]:
print("\nWEEKLY TOTALS\n")

for j in departments:
    weekly_ors = sum(x[(j, k)].value() for k in days)
    weekly_hours = hours_per_OR * weekly_ors

    print(f"{j:18s} | ORs/week: {int(weekly_ors):2d} | Hours/week: {weekly_hours:5.1f}")



WEEKLY TOTALS

Ophthalmology      | ORs/week:  6 | Hours/week:  48.0
Gynecology         | ORs/week: 15 | Hours/week: 120.0
Oral Surgery       | ORs/week:  2 | Hours/week:  16.0
Otolaryngology     | ORs/week:  4 | Hours/week:  32.0
General Surgery    | ORs/week: 23 | Hours/week: 184.0
